In [1]:
#The path to the biopython packages was not automatically added to anaconda. To allow import of the package, a manual path must be specified.
import os
#Absolute location: 

#os.environ['CONDA_PREFIX']=/home/wish1832/anaconda3
#Code to append biopython location to path
os.environ['PATH'] = os.environ['PATH']+';'+os.environ['CONDA_PREFIX']+"/pkgs/biopython-1.78-py38h7b6447c_0/lib/python3.8/site-packages"

#### Based on analysis in R, Uniq2 (alignment[1]) contains the reference sequence. 

# Scaffolding Code for Variant Calling

## Functions

In [2]:
from Bio import AlignIO as msa_in
from Bio.Align import MultipleSeqAlignment as msa
import Bio.SeqRecord
import numpy as np
import pandas as pd

def initalize_file(outname):
    """
    outname = the directory and prefix of the output file (string)
        For example, setting outname "mydir/mutant" will create an outfile in the directory mydir with the name mutant_variants_raw.tsv.
        Other reports generated by the output will also exist in the directory mydir and have the prefix "mutant".
    
    Given an output file prefix, initialize_files() will create one file (outname_variants_raw.tsv) 
    containing raw data on the substitutions, insertions, and deletions relative to the reference genome found in the sequence alignment. 
    
    The function will print headers in each file in the order specified below:
    
    Field 1: Cluster_ID (e.g. clust48)  
    Field 2: Number of sequences in cluster 
    Field 3: Type of variant (insertion, deletion, substitution)
    Field 4: Mutation code (formatted based on the nomenclature indicated by the Human Genome Variation Society https://varnomen.hgvs.org/recommendations/protein/)
    Field 5: Amino Acid(s) of Reference Strand 
        For insertions, this is the two amino acid residues the insertion ocurrs between.
        For deletions, this will be the reference residue(s) deleted 
    Field 6: Variant Amino Acid Residue
        For multi-codon insertions, this field will contain a string of multiple codons
        For deletions, this will be blank
    Field 7: Start position of variant on multiple sequence alignment
        For insertions this will be the codon number on the multiple sequence alignment before the inserted codon(s)
    Field 8: End position of variant on multiple sequence alignment 
        For insertions this will be the codon number on the multiple sequence alignment before the inserted codon(s)
        For substitutions and single-base deletions this will be blank
    Field 9: Start position of variant on reference strand 
        For insertions this will be the codon number on the reference strand before the inserted codon(s)
    Field 10: End position of variant on reference strand
        For insertions this will be the codon number on the reference strand before the inserted codon(s)
        For substitutions and single-base deletions this will be blank
    """
    #Create filename for raw output
    raw_out = outname+"_variants_raw.tsv"
    #Define header according to elements listed in docstring
    header = ["Cluster_ID","Cluster_Size","Type","Code","Ref Residue(s)","Var Residue(s)","AA_Start(MSA)","AA_End(MSA)","AA_Start(Ref)","AA_End(Ref)"]

    #Print header in output file
    with open(raw_out,"w") as outfile:
        outfile.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(header[0],header[1],header[2],header[3],header[4],header[5],header[6],header[7],header[8],header[9]))

    """Code below will prevent overwriting in final version.
        if not file:
            with open(file,"w") as file
        else
        
            print("Error: {} already exists. Overwriting prohibited.".format(file))"""
    #Return the filename for use in other functions
    return raw_out

def write_line(outfile,clustname,clustsize,var_type,code,ref_AA,var_AA,msa_start,ref_start,msa_end="-",ref_end="-"):
    """
    Will write out to the variant .tsv values given the location of an output list with 6 Fields in the following format:
    
    Field 1: Cluster_ID (e.g. clust48)  
    Field 2: Number of sequences in cluster 
    Field 3: Type of variant (insertion, deletion, substitution)
    Field 4: Mutation code (formatted based on the nomenclature indicated by the Human Genome Variation Society https://varnomen.hgvs.org/recommendations/protein/)
    Field 5: Amino Acid(s) of Reference Strand 
        For insertions, this is the two amino acid residues the insertion ocurrs between.
        For deletions, this will be the reference residue(s) deleted 
    Field 6: Variant Amino Acid Residue
        For multi-codon insertions, this field will contain multiple columns
        For deletions, this will be blank
    Field 7: Start position of variant on multiple sequence alignment
        For insertions this will be the codon number on the multiple sequence alignment before the inserted codon(s)
    Field 8: End position of variant on multiple sequence alignment 
        For insertions this will be the codon number on the multiple sequence alignment before the inserted codon(s)
        For substitutions and single-base deletions this will be blank
    Field 9: Start position of variant on reference strand 
        For insertions this will be the codon number on the reference strand before the inserted codon(s)
    Field 10: End position of variant on reference strand
        For insertions this will be the codon number on the reference strand before the inserted codon(s)
        For substitutions and single-base deletions this will be blank
    """
    #Write to output file
    if outfile:
        with open(outfile,"a") as file:
            file.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format(clustname,clustsize,var_type,code,ref_AA,var_AA,msa_start,msa_end,ref_start,ref_end))
    
    #Raise a FileNotFoundError if the output file has not been initialized
    else:
        raise FileNotFoundError("{} does not exist.".format(outfile))        

def reference_position(reference):
    """
    For Variant Code Creation, the positions must correspond to the reference sequence, rather than the alignent.
    
    Given the reference genome, reference_position() will return a list of integers that stores the reference genome position corresponding to the msa position.
    If there is no base in the reference genome at a given position in the msa (when a "-" is in the reference genome), a blank is stored in this list.
    """
    ref_list=[]
    pos=1
    for i in reference:
        if i != "-":
            ref_list.append(pos)
            pos += 1
        else:
            ref_list.append("-")
    return ref_list        
        
def msa_iterator1(alignment,raw_out,reference,ref_index,outfile):
    """
    alignment=a biopython multiple sequence alignment object
    raw_out=filename for outputting raw data
    reference=Reference strand to which sequences are compared (biopython RefSeq object)
    ref_list=A list relating an amino acid position on a multiple sequence alingment to its corresponding position on the reference strand
    i_begin=Amino acid position on multiple sequence alignment at which to begin iteration (allows for skipping over multi-base insertions and deletions)
    j_begin=Alignment object at which to begin
    
    
    msa_iterator() will iterate through the multiple sequence alignment object and compare bases to the reference genome. 
    Insertions, deletions, and substitutions will be detected and handled by separate functions based on type. 
    
    i is the amino acid position, j is the sequence position in the MSA (origional iterated through AA position first)
    """
    #Every sequence in the alignment will be iteratively compared to the reference strand
    #Multi-base deletions and insertions require a loop in which iterables can be edited
    #A while loop will be used instead of a for loop to iterate through all i codons in j genomes.
    
    #Begin with i and j equal to zero
    i,j=0,0
    
    #While loop will run until the last i position of the last j strain is tested.
    while not (j==len(alignment)-1 and i==len(reference)):
        #When all i codons in a genome have been iterated through, the genome index is advanced by one and the i is reset to zero.
        if i==len(reference):
            j+=1
            i=0
        else:
            #For every codon, test the following:
            print("testing j={}, i={}".format(j,i))
            #Conditional to detect a substitution
            if reference[i] != alignment[j,i] and reference[i]!="-" and alignment[j,i]!="-":
                print("AA substitution: position {0} in cluster {3}: {1}{0}{2}".format(i+1,reference[i],alignment[j,i],j+1))
                substitution_case(i,j,alignment,reference,ref_index,outfile)
                #Move on to next codon
                i+=1

            #Conditional to detect a deletion
            elif (reference[i] != alignment[j,i]) and alignment[j,i]=="-":
                print("AA deletion: position {0} in cluster {3}: {1}{0}{2}".format(i+1,reference[i],alignment[j,i],j+1))
                print("j={}, i={}".format(j,i))
                #Call the deletion_case function, which will check for a multi-base deletion and return the value i to resume iteration from 
                i=deletion_case(j,i,alignment,reference,ref_index,outfile)

            #Conditional to detect an insertion
            elif reference[i] != alignment[j,i] and reference[i]=="-":
                print("AA insertion: position {0} in cluster {3}: {1}{0}{2}".format(i+1,reference[i],alignment[j,i],j+1))
                i=insertion_case(j,i,alignment,reference,ref_index,outfile)
                #insertion_case will set i to the next codon automatically
            
            #If there is no variant, increase i by one to test the next codon.
            elif reference[i]==alignment[j,i]:
                i+=1
            
            #Test for unanticipated scenarios
            else:
                print("A codon comparison has been found that does not meet any conditionals in the msa_iterator() loop.")
            
    print("The while loop has been broken.")
        
def substitution_case(i,j,alignment,reference,ref_index,outfile):
    """
    Will activate when a substution is detected during iteration through the multiple sequence alignment. 
    """
    #Store position of variant in MSA (codon position is one more than the index)
    msa_pos=i+1
    #Store position of variant in reference genome
    ref_pos=ref_index[i]
    #Store codon in position i of reference genome
    ref_AA=reference[i]
    #Store the variant codon
    var_AA=alignment[j,i] #Retrieves the codon from the sequence at position j, at position i
    #Store the name of the cluster associated with the variant sequence (sequence index j)
    #The cluster name is in the id of the RefSeq object, before the ";".
    clustname=alignment[j].id.split(sep=";")[0]
    #Retrieve size of cluster associated with the variant sequence j
    #The cluster size is printed betweeen ";" and "=" in id
    clustsize=alignment[j].id.split(sep="=")[1].split(sep=";")[0]
    #Store the substitution code
    #Based on the sequence variant nomenclature suggested by the Human Genome Variation Society,
    #The code format is <Reference_Amino_Acid><Codon_Position(Reference Genome)><Variant_Amino_Acid>
    #Source (https://varnomen.hgvs.org/recommendations/protein/variant/substitution/)
    code="{}{}{}".format(ref_AA,ref_pos,var_AA)
    
    #Write above information to file
    write_line(outfile,clustname,clustsize,"sub",code,ref_AA,var_AA,msa_pos,ref_pos,"-","-")
    
    #Once information is stored, return to iteration at the next base (returns to the For loop in the parent function)

def deletion_case(j,i,alignment,reference,ref_index,outfile):
    """
    Will activate when a deletion is detected during iteration through the multiple sequence alignment. 
    
    Since deletions may span two or more bases and nomenclature counts multi-base deletions as a single event, 
    iteration through the multiple sequence alignment in the parent msa_iterator() function must be skipped ahead
    to the end of the detion region to avoid a multiple-codon deletion being read as multiple instances of a single-base deletion.
    After recording a deletion, msa_iterator() is restarted at the position i corresponding to the codon after the end of the deletion. 
    """
    #Raise an error if the function is triggered by a codon that is not deleted
    if not (alignment[j,i]=="-" and reference[i]!="-"):
        raise ValueError("Deletion case called on a codon region without a deletion")
    #Or if the deletion case is not called at the beginning of a deletion region
    #Code detects a deletion in the codon one index before the current i it is called on
    elif i!=0 and ((reference[i-1]!=alignment[j,i-1]) and alignment[j,i-1]=="-"): 
        raise ValueError("Deletion case called in the middle of a multi-base deletion")

    #Record the position at the beginning of a deletion
    i_start=i   
   
    #Step 1: Iterate forward until extant codons are found on both the reference strand and genome j.
    while True:
        print("Testing i={}".format(i))
        #If a deletion case is detected, or if both the reference genome and genome j have a blank at index i, try the next codon.
        if ((alignment[j,i]=="-" and reference[i]!="-") or (alignment[j,i]=="-" and reference[i]=="-")):
            print("Skipping blank at position {}".format(i+1))
            #If the base currently being tested at the end of the genome, a special case is triggered 
            if i==len(reference)-1:
                print("Special case: deletion extends to end of genome.")
                #Set i_Cflank as being non-existent (affects which index is returned to msa_iterator() at end of function)
                i_Cflank=None
                break #Break the while loop and continue to step 2.
            else:
                i+=1 #Increase index by one and continue to test the next codon 
        
        #When the C-flanking extant codon is found, record the index of the flank.
        elif (alignment[j,i]!="-" and reference[i]!="-"):
            print("Found C-terminal flanking codon at index {}.".format(i))
            #The index of the C-flanking codon will be returned to msa_iterator() as the position to resume variant checking.
            i_Cflank=i
            #Break the while loop and continue to step 2.
            break 
            
        #Special case: it is possible for an insertion to ocurr with a deletion (delins event).
        elif alignment[j,i]!="-" and reference[i]=="-":
            print("Found a delins event.")
            #Run the delins case and store the index of the C-flanking codon to the event
            i=delins_case(j,i_start,alignment,reference,ref_index,outfile)
            #msa_iterator() will continue testing for variants at the index i
            return i

    #Step 2: Iterate backwards over any blanks that may exist until the end of the deletion region is reached.
    print("Iterating backwards to find i_end.")
    print("Range of iterable",range((i_Cflank if i_Cflank else len(reference)-1),i_start-1,-1),sep=";")
    #Iteration will begin at the C-flanking codon (or the end of the genome) and will include the start of the deletion region.
    for i in range((i_Cflank if i_Cflank else len(reference)-1),i_start-1,-1):
        print("Testing i={}.".format(i))
        #If a blank is discovered at the position, continue to previous codon.
        if (reference[i]=="-" and alignment[j,i]=="-"):
            print("Skipping backwards over blank.")
        
        #If a deletion is found, stop iteration and record current position as i_end (end of deletion region).
        elif (reference[i]!="-" and alignment[j,i]=="-"):
            print("Discovered end of indel region. Recording index.")
            i_end=i
            print("i_end=",i_end,sep=" ")
            #Break the for loop and continue to step 3.
            break
    
    #Step 3: Record deletion information using i_start and i_end.  
    print("recording deletion info.")
    record_deletion_info(j,i_start,i_end,alignment,reference,ref_index,outfile)
    
    #Return the index of the C-flanking codon to msa_iterator() if it exists. 
    #For a deletion at the end of the genome, return i=len(reference), which will cause msa_iterator() to skip to the next line.
    return (i_Cflank if i_Cflank else len(reference))
        
def insertion_case(j,i,alignment,reference,ref_index,outfile):
    """
    insertion_case() will gather the following information regarding the insertion in 5 steps:
    
    instype=Either "ins", "N_ext", or "C_ext". instype will tell the function record_insertion_info() which
        type of insertion case to record. Insertions at the 5' end of the transcriptome are N-terminal extentions,
        and must be recorded differently from standard insertions. The same is true of C-terminal extentions, which
        are insertions at the 3' end of the transcriptome.
        
        If N-terminal or C-terminal extentions have not been discovered, instype will be set to "ins".
        
    i_start=The index of the start of the insertion region (first location where reference is blank and the variant genome has a codon)
    
    i_Nflank= The index of the flanking N-terminal codon to the insertion in the reference strand (required for the code).
        For N-terminal extentions, this will be equal to "None".
    
    i_end=The index of the end of the insertion region (last location where the reference is blank and the variant genome has a codon)
    
    i_Cflank=The index of the flanking C-terminal codon to the insertion in the reference strand (required for the insertion code).
        For C-terminal extentions, this will be equal to "None".

    """
    instype=None
    
    #Raise an error if the function is triggered by a codon that is not inserted
    if not (alignment[j,i]!="-" and reference[i]=="-"):
        raise ValueError("Deletion case called on a codon region without a insertion")
    #Or if the insertion case is not called at the beginning of a deletion region
    #Code detects a insertion in the codon one index before the current i it is called on
    elif i!=0 and ((reference[i-1]!=alignment[j,i-1]) and reference[i-1]=="-"): 
        raise ValueError("Deletion case called in the middle of a multi-base insertion")
    
    #Step 1: Record the index at beginning of insertion
    i_start=i
    
    #Begin iteration
    #Step 2: It is not necessarily true that the flanking reference codon is one index before the insertion start.
    #A for loop will iterate backwards until an extant reference codon is found. 
    #If the insertion region is discovered in the first codon position, a special case is triggered.
    if i!=0:
        for idx in range(i-1,-1,-1): #Iterates backwards from the start of the insertion region, ending at 0
            #idx used instead of i to prevent overwriting of i to be passed to be msa_iterator
            if reference[idx]=="-":
                print("Iterating backward over blank.")
                #Special case, if an N-terminal flanking codon has not been discovered when the beginning of the genome
                #is reached, this is an N-terminal extention.
                if idx==0:
                    print("Insertion happens before start of reference sequence. This is an N-terminal extention.")
                    i_Nflank=None
                    break #Break the loop and continue to step 3.
            
            #If the N-terminal flanking codon is found, store the index of the codon.
            elif reference[idx]!="-":
                print("Found N-terminal flanking codon")
                i_Nflank=idx
                break #Break the loop and continue to step 3.
    
    #Step 2: Special case of insertion at beginning of region 
    #Called when an insertion is found starting at i=0 (impossible for a N-terminal flank to exist)
    else:
        print("Special case: N-extention triggered.")
        i_Nflank=None
        
    #Step 3: Iterate forward until the first position of an extant codon on both the reference and the variant strand 
    #Will yield i_Cflank 
    print("Beginning forward iteration to find i_Cflank.")
    while True: 
        print("Testing j={}, i={}".format(j,i))
    
        #Check for an insertion, or a blank on both the reference and the genome j, at the codon position. If so, continue on
        if (reference[i]=="-" and alignment[j,i]!="-") or (reference[i]=="-" and alignment[j,i]=="-"):
            #Special case: end of the genome is reached and contains a blank (will also be triggered if the 
            #insertion begins at the end of the variant genome). This is a C-terminal extension.
            if i==len(reference)-1:
                print("End of genome reached. C-extension special case triggered")
                i_Cflank=None
                break #Terminate the loop and continue to step 4  
            
            #If not at the end of the genome, increase the index by one and test the next codon.
            else:
                print("Blank at position {}".format(i+1))
                i+=1

        #Check for a deletion at the codon. If so, switch to the indel case
        elif reference[i]!="-" and alignment[j,i]=="-":
            print("Indel case found.")
            #i_start will be passed to the indel function
            i=delins_case(j,i_start,alignment,reference,ref_index,outfile)
            return i #After delins case is ran, return the position of the C-terminal flank if it exists to the msa_iterator() function

        #Check for extant codons in both reference and genome j at the position
        elif reference[i]!="-" and alignment[j,i]!="-":
            #The position of the C-terminal flanking codon will be stored.
            print("Reached C-flanking codon.")
            i_Cflank=i
            break #Terminate the loop and continue to step 4.
    
    #Step 4: Iterate backward over any blank codons on both the reference and variant strands until a codon on the variant strand is reached
    #Iteration will begin one codon before the C-terminal flank if it exists, otherwise iteration begins at the end of the genome.
    print("Iterating backwards to discover the end of the insertion region.")
    for idx in range((i_Cflank-1 if i_Cflank else len(reference)-1),(i_Nflank if i_Nflank else -1),-1): #Iterates backwards to i_Nflank (excluding i_Nflank) if it exists, otherwise iterates to zero(inclusive).
        print("Testing i={}".format(idx))
        #if a blank is found, continue backwards iteration 
        if alignment[j,idx]=="-":
            print("Skipping backwards over blank.")
            continue
        
        #The first codon found in the alignment will mark the end of the inserted sequence in the multiple sequence alignment.
        elif alignment[j,idx]!="-":
            print("The insertion region ends at position i={}. The ending codon is {}.".format(idx,alignment[j,idx]))
            i_end=idx
            break #Break the loop and continue to step 5.
    
    #Step 5: determine type of insertion based on the presence/absence of N-terminal and C-terminal flanking codons
    if i_Cflank and i_Nflank:
        print("Insertion has both C- and N- terminal flanks.")
        instype="ins"
    elif i_Cflank and not i_Nflank:
        print("Insertion has only a C-flanking codon and is therefore an N-terminal extension.")
        instype="N_ext"
    elif i_Nflank and not i_Cflank:
        print("Insertion has only an N-flanking codon and is therefore an N-terminal extension.")
        instype="C_ext"
        
    print("The insertion type interpereted is {}".format(instype))
    
    #All indeces have been gathered. The indeces will be passed to the record_insertion_info() function to record the insertion in the file
    record_insertion_info(j,instype,i_start,i_end,alignment,reference,ref_index,outfile,i_Nflank,i_Cflank)
    #Recording complete. Pass index of the C-flanking codon back to msa_iterator() if it exists 
    return (i_Cflank if i_Cflank else len(reference))

def delins_case(j,i_start,alignment,reference,ref_index,outfile):
    """
    insertion_case() will gather the following information regarding the insertion-deletion (indel) event in 5 steps:
    
    i_start: The beginning of the insertion region. This is passed to the function by the function that triggered the delins case.
    
    i_end: The index of the end of the indel region (last location where the reference is blank and the variant genome has a codon)
    
    i_first: The index of the first codon deleted (used for computation of the variant code)
    
    i_last: The index of the last codon deleted (used for computation of the variant code)
    
    codons_inserted: The amino acid codons that have been inserted into the deletion region, with any blanks between residues removed.  
    """
    #Step 1: Iterate until the C-terminal codon of the delins region is found
    #This will be when there is an extant codon on both the reference strain and on genome j.
    i=i_start
    print("Iterating forward until the end the delins region is reached.")
    while True:
        #Test the current codon.
        print("Testing i={}".format(i))
        #If the current codon is an insertion, deletion, or a double blank, continue iteration.
        if (reference[i]!="-" and alignment[j,i]=="-") or (reference[i]=="-" and alignment[j,i]!="-") or (reference[i]=="-" and alignment[j,i]=="-"):
            print("The current codon is within the indel region. Continuing.")
            #Special case: the end of the genome is reached before a C-terminal flank is found
            if i==len(reference)-1:
                print("Reached the end of the loop without finding a C-terminal flank.")
                i_Cflank=None
                #Break the while loop and continue to step 2
                break
            else:
                i+=1
                
        #If there are codons on the reference and alignment genomes at the current position, the end of the indel region has been reached.
        elif (reference[i]!="-" and alignment[j,i]!="-"):
            print("Reached the C-terminal flanking codon to the indel region.")
            #Store this position as the position of the C-terminal flanking codon to the indel region (iteration in msa_iterator() must resume from this point.) 
            i_Cflank=i
            break #Break the while loop and continue to step 2.
            
    #Step 2: Iterate backward over any blanks that may exist until the last codon that is either inserted or deleted
    #Record this position as i_end
    print("Iterating backward to the end of the indel region.")
    for i in range((i_Cflank if i_Cflank else len(reference)-1),i_start-1,-1):
        print("Testing i={}".format(i))
        #If a blank is discovered at the position, continue to previous codon.
        if (reference[i]=="-" and alignment[j,i]=="-"):
            print("Skipping backwards over blank.")
            continue
        
        #If a deletion or an insertion is found, stop iteration and record current position as i_end (end of indel region).
        elif (reference[i]!="-" and alignment[j,i]=="-") or (reference[i]=="-" and alignment[j,i]!="-"):
            print("Discovered end of indel region. Recording index.")
            i_end=i
            break
    
    #Step 3: Iterate through the indel region on the reference strand, storing the indeces of the first and last codons
    #The first and last codons on the reference strand are the first and last amino acids deleted (necessary for code).
    #Iteration is through region from i_start up to and including i_end.
    
    #Step 3.1: Finding the first codon
    for i in range(i_start,i_end+1):
        if (reference[i]=="-"):
            continue
        elif (reference[i]!="-"):
            print("Found the first codon of the reference strand (i={}).".format(i))
            #Store the index of the first codon
            i_first=i
            break
    
    #Step 3.2: Finding the last codon
    for i in range(i_end,i_start-1,-1):
        if (reference[i]=="-"):
            continue
        elif (reference[i]!="-"):
            print("Found the last codon of the reference strand (i={}).".format(i))
            #Store the index of the last codon
            i_last=i
            break
            
    #Step 4: Iterate through the slice of the variant strand and store the codons, removing any blanks 
    #The function insertion_var_AA() will do this.
    codons_inserted=AA_blank_remover(i_start,i_end,j,alignment)
    
    #Step 5: Record information using the indeces discovered.
    record_delins_info(i_start,i_end,i_first,i_last,j,codons_inserted,alignment,reference,ref_index,outfile)
    
    #Must return the position of the C-flanking codon to the indel region, so msa_iterator() does not record the region more than once.
    return (i_Cflank if i_Cflank else len(reference)) #If the end of the genome is reached, pass that index to msa_iterator().
    
def record_deletion_info(j,i_start,i_end,alignment,reference,ref_index,outfile):
    """
    Information recording
    """
    #Store the name of the cluster associated with the variant sequence (sequence index j)
    #The cluster name is in the id of the RefSeq object, before the ";".
    clustname=alignment[j].id.split(sep=";")[0]
    #Retrieve size of cluster associated with the variant sequence j
    #The cluster size is printed betweeen ";" and "=" in id
    clustsize=alignment[j].id.split(sep="=")[1].split(sep=";")[0]

    #Conditional: for single codon deletions, positions and amino acid residues will be stored as a single entry
    #For multi-codon deletions, data will be stored as a range
    #For single-codon deletions
    if i_start==i_end:
        #Store position of variant in MSA
        msa_start=i_end+1
        msa_end="-"
        #Store position of variant in reference genome
        ref_start=ref_index[i_end]
        ref_end="-"
        #Store codon in position i of reference genome
        ref_AA=reference[i_end]
        #Store the variant codon
        var_AA="-" #Retrieves the codon from the sequence at position j, at position i    
        #Store the deletion code
        """
        Based on the sequence variant nomenclature suggested by the Human Genome Variation Society,
        the code format is <Amino_Acid_Deleted><Position_of_Deleted_Codon, reference Genome>del
        source: (https://varnomen.hgvs.org/recommendations/protein/variant/deletion/)
        """
        code="{}{}del".format(ref_AA,ref_start)
    #For multi-codon deletions
    else:
        #Store position range in MSA
        msa_start=i_start+1
        msa_end=i_end+1

        #Store position range in reference genome
        ref_start=ref_index[i_start]
        ref_end=ref_index[i_end]

        #Store reference amino acids that have been deleted in sequence j
        #Slicing does not include the end value, so the end value should be one index after the end of the deletion region (i)
        ref_AA=deletion_ref_AA(i_start,i_end,reference)

        print(ref_AA)
        #Store a blank for deleted variant codons
        var_AA="-"
        print(var_AA)
        #Store the deletion code
        """
        Based on the sequence variant nomenclature suggested by the Human Genome Variation Society,
        the code format is <Beginning_Amino_Acid><Beginning_Codon_Position, reference Genome>_<Ending_Amino_Acid><Ending_Codon_Position>del
        source: (https://varnomen.hgvs.org/recommendations/protein/variant/deletion/)
        """
        code="{}{}_{}{}del".format(reference[i_start],ref_index[i_start],reference[i_end],ref_index[i_end])
    
    #Write above information to file
    write_line(outfile,clustname,clustsize,"del",code,ref_AA,var_AA,msa_start,ref_start,msa_end,ref_end)
    
def record_insertion_info(j,instype,i_start,i_end,alignment,reference,ref_index,outfile,i_Nflank=None,i_Cflank=None):
    #Store the name of the cluster associated with the variant sequence (sequence index j)
    #The cluster name is in the id of the RefSeq object, before the ";".
    clustname=alignment[j].id.split(sep=";")[0]
    #Retrieve size of cluster associated with the variant sequence j
    #The cluster size is printed betweeen ";" and "=" in id
    clustsize=alignment[j].id.split(sep="=")[1].split(sep=";")[0]
    
    #Store starting position on the multiple sequence alignment
    msa_start=i_start+1
    ref_AA="-"
    
    #msa_end and var_AA: depend on whether the insertion is a single base or a multi-base insertion 
    if i_start==i_end:
        msa_end="-"
        var_AA=alignment[j,i_start]
    else:
        msa_end=i_end+1
        var_AA=AA_blank_remover(i_start,i_end,j,alignment)
    
    #ref_start and ref_end: defined as the reference positions of the N- and C- terminal flanking codons if they exist
    ref_end=(ref_index[i_Cflank] if i_Cflank else "-") #C-flank is the flank downstream of the insertion.
    ref_start=(ref_index[i_Nflank] if i_Nflank else "-")
    
    #The flanking reference residues are stored for creation of the code, if they exist.
    Nflank=(reference[i_Nflank] if i_Nflank else None)
    Cflank=(reference[i_Cflank] if i_Cflank else None)
 
    
    #The "type" and code fields depend on whether the insertion is a normal case, an N-terminal extension, or a C-terminal extension.
    #TODO: instype will be passed through for output. Delete this later.
    if instype=="ins":
        print_type=instype
        #Code is formatted as <N_flank amino acid ><>_<><>ins<var_AA>
        code="{}{}_{}{}ins{}".format(Nflank,ref_start,Cflank,ref_end,var_AA)
        
    elif instype=="N_ext":
        #print type stores the type to be printed to the output file, as well as the type indicated in the code
        #Both N-terminal and C-terminal extentions will be represented as "ext"
        print_type="ext"
        #Code: will be formatted <C-flanking amino acid>1ext<Var_AA inserted>
        #TODO: This is at odds with the Human Genome Variation Society, which reccomends that extentions describe the number of new codons instead of their type.
        #I chose to print the type because we don't have the genetic information in this experiment. Should I keep things this way or change the format to match the reccomendations?
        code="{}1ext{}".format(Cflank,var_AA)
        
    elif instype=="C_ext":
        print_type="ext"
        #Code: will be formatted <N-flanking amino acid><reference position of N-flanking amino acid>ext<Var_AA inserted>
        code="{}{}ext{}".format(Nflank,ref_start,var_AA)
    
    #Write infomation to file 
    write_line(outfile,clustname,clustsize,print_type,code,ref_AA,var_AA,msa_start,ref_start,msa_end,ref_end)
    
def record_delins_info(i_start,i_end,i_first,i_last,j,codons_inserted,alignment,reference,ref_index,outfile):
    ###Record cluster information###
    #Store the name of the cluster associated with the variant sequence (sequence index j)
    #The cluster name is in the id of the RefSeq object, before the ";".
    clustname=alignment[j].id.split(sep=";")[0]
    #Retrieve size of cluster associated with the variant sequence j
    #The cluster size is printed betweeen ";" and "=" in id
    clustsize=alignment[j].id.split(sep="=")[1].split(sep=";")[0]
    
    #Record type (always equal to delins)
    print_type="delins"
    
    ###Record indel code###
    #Format: <first codon deleted><position of first codon deleted>_<last codon deleted><position of last codon deeted>delins<AA codons inserted>
    
    #Store identity and position of first codon deleted, using its index
    del_AA_first=reference[i_first]
    #ref_start=Position of first codon deleted
    ref_start=ref_index[i_first]
    
    #Code format depends on if there is one or more codons deleted. 
    #If multiple codons are deleted (i_first!=i_last), calculate the code in the format below.
    if i_first!=i_last:
        #Store identity and position of last codon deleted, using its index
        del_AA_last=reference[i_last]
        #ref_end=Position of last codon deleted
        ref_end=ref_index[i_last]
        #Indel code
        code="{}{}_{}{}delins{}".format(del_AA_first,ref_start,del_AA_last,ref_end,codons_inserted)
    #Follow the conditional below for single-codon deletions.
    else:
        ref_end="-" #will store a blank on the raw data output file.
        code="{}{}delins{}".format(del_AA_first,ref_start,codons_inserted)
        
    ###Record AA sequence on reference and variant strands###
    #ref_AA and var_AA will simply be the reference sequence and the variant sequence in the indel region,
    #Without blanks removed.
    ref_AA=str(reference[i_start:i_end+1])
    var_AA=str(alignment[j,i_start:i_end+1].seq)
    
    #Record start and end positions on multiple sequence alignment (equal to index of start and end of region plus one)
    msa_start=i_start+1
    msa_end=i_end+1
    
    #Write information to file
    write_line(outfile,clustname,clustsize,print_type,code,ref_AA,var_AA,msa_start,ref_start,msa_end,ref_end)
    
def deletion_ref_AA(i_start,i_end,reference):
    """
    For multi-base deletions, records the refernce AA's deleted without blanks that may appear in the aligned reference genome.
    
    This function should only be called on multi-base deletions, otherwise an error will occur
    """
    ref_AA=reference[i_start]
    #the amino acid at i_start is included first, and iteration goes up to and including i_end (must be i_end+1 because range() does not include stop value)
    for i in range(i_start+1,i_end+1):
        #If an amino acid exists on the reference genome (no blank on msa), add the AA to the sequence
        if reference[i]!="-":
            ref_AA=ref_AA+reference[i]
    return ref_AA

def AA_blank_remover(i_start,i_end,j,alignment):
    """
    AA_blank_remover() records the amino acids within a specified region on a specified genome j, 
    excluding blanks that may exist due to the multiple sequence alignment.
    
    For example, an insertion reading "FLP--AV" in the multiple sequence alginment will be converted to "FLPAV".
    
    i_start: the starting index of the region (integer)
    i_end: the ending index of the region (integer)
    j: the index of the genome to be read (integer)
    alingment: a biopython multiple sequence alignent object consisting of biopython SeqRecord objects.
    """
    #Begin with an empty variable 
    AA_Seq=None
    #Iterate from the first codon in the region to the last, including both first and last
    for i in range(i_start,i_end+1):
        #Blanks will be skipped by the conditionals below.
        #The first non-blank character will be detected and assigned to AA_seq
        if alignment[j,i]!="-" and not AA_Seq:
            AA_Seq=alignment[j,i]
        #If there is at least one codon already stored in AA_seq, the next codon will be appended to the existing sequence.
        elif alignment[j,i]!="-" and AA_Seq:
            AA_Seq=AA_Seq+alignment[j,i]
    return AA_Seq
                
"""
Part 2: Functions for analysis of raw variant data and report production
"""



def initialize_report_files(outname):
    """
    outname = the directory and file prefix of the output files. Specific files will contain 
    
    intitialize_report_files will create four files containing reports on the mutations present.
    subs_report,ins_report,and dels_report will contain files
    """
    subs_report=outname+"_substitutions.txt"
    ins_report=outname+"_insertions.txt"
    dels_report=outname+"_deletions.txt"
    clusters_report=outname+"_clusters.txt"
    
    #Create blank files to be filled with information in later steps 
    for file in [subs_report,ins_report,dels_report,clusters_report]:
        with (open(file,"w")) as outfile:
                outfile.write("")
    return subs_report,ins_report,dels_report,clusters_report



In [51]:
str(reference[3:6])

'--L'

In [55]:
str(alignment[11,3:6].seq)

'--L'

In [4]:
def output_clusters_by_code(df,outfile):
    #Iterator: loops through all unique codes 
    for i in range(len(df.code.unique())):
        sub_df = df[df.code==df.code.unique()[i]]
        #Create empty lists to store all cluster IDs with the variant in question, and the size of each cluster
        clusters=[]
        clust_sizes=[]
        #Loop through all entries in the dataframe that match the current cluster ID
        for item in sub_df["Cluster_ID"]:
            #Add the cluster ID to the cluster list
            clusters.append(item)
            #Find the size of the cluster ID and add that to the list for cluster sizes
            clust_sizes.append(sub_df.loc[sub_df["Cluster_ID"]==item,"N_sequences_in_cluster"].iat[0])
        #Once the lists for the cluster IDs and sizes for the variant type have been created, iterate through each list and add it to the report file.
        #First, the total number of sequences with the given variant is computed and added to the file next to the variant code (variant header).
        total_seq=np.array(clust_sizes).sum()
        #Write variant header information to file
        with (open(outfile,"a")) as file:
            file.write(">{}|{} sequences".format(df.code.unique()[i],total_seq))
            #Iterate through the cluster names and sizes in the lists and add information to file
            for j in range(len(clusters)):
                file.write(clusters[j]," size "+str(clust_sizes[j]),sep=":")
            #Add a line break at the end of the information for the current code
            file.write("\n")

(array([], dtype=int64),)

In [185]:
ref_index

['-', '-', '-', '-', '-', 1, 2, '-', '-', 3, 4, 5, 6, 7, '-', '-']

## Program Execution

In [ ]:
"""
Fake test AA sequence to check function of program
"""
#Load MSA, store as biopython multiple sequence alingment object
alignment = msa_in.read("msa_tester.fasta","fasta")

#Initialize files
#Define output file name
outname = "../Variant_Call/test_tester"
raw_out = initalize_file(outname)

#Define reference genome (here it is index 1 of the multiple sequence alignment)
reference=alignment[0].seq
#Create reference sequence object storing reference sequence indeces corresponding to msa index (if "-" in reference seq after alignment)
ref_index=reference_position(reference)

#Iterate through multiple sequence alignment and write raw data from variant information to file
msa_iterator1(alignment,raw_out,reference,ref_index,raw_out)

In [ ]:
"""
Part 1: Generate raw variant data from multiple sequence alignment input
"""
#Load MSA, store as biopython multiple sequence alingment object
alignment = msa_in.read("../Aligned_Sequences/spikeprot0902_full_aligned_nosingletons_1iter.fasta",'fasta')

#Initialize files
#Define output file name
outname = "../Variant_Call/test"
raw_out = initalize_file(outname)

#Define reference genome (here it is index 1 of the multiple sequence alignment)
reference=alignment[1].seq
#Create reference sequence object storing reference sequence indeces corresponding to msa index (if "-" in reference seq after alignment)
ref_index=reference_position(reference)

#Iterate through multiple sequence alignment and write raw data from variant information to file
msa_iterator1(alignment,raw_out,reference,ref_index,raw_out)

In [4]:
#Step 2: Report information from raw file
variants = pd.read_csv(raw_out,delimiter="\t")

#Initialize output files
subs_report,ins_report,dels_report,clusters_report=initialize_report_files(outname)

"""
Reporting Substitutions
"""



In [21]:
#Code for a multi-level filter
sample=variants[(variants["Type"]=="del")&(variants["Cluster_ID"]=="Uniq1745")]

In [41]:
len(alignment[0])

1320

In [48]:
alignment[734]

SeqRecord(seq=Seq('-FFFFLHFALVNSQCVNLTGRAAIQPSFTNSSQRGVYYPDTIFRSNTLVLSQGY...YT-'), id='Uniq735;size=4;', name='Uniq735;size=4;', description='Uniq735;size=4;', dbxrefs=[])

In [22]:
sample

,Type,Nuc_Pos(MSA),Nuc_Pos(Ref),Ref Residue,Var Residue,Cluster_ID,N_sequences_in_cluster
56,del,1,1,M,-,Uniq1745,2
139,del,2,2,F,-,Uniq1745,2
221,del,3,3,V,-,Uniq1745,2
306,del,4,4,F,-,Uniq1745,2
418,del,5,5,L,-,Uniq1745,2
511,del,6,6,V,-,Uniq1745,2
594,del,7,7,L,-,Uniq1745,2
680,del,8,8,L,-,Uniq1745,2
766,del,9,9,P,-,Uniq1745,2
846,del,10,10,L,-,Uniq1745,2


In [26]:
if len(sample[(sample["Nuc_Pos(Ref)"]==17)])==0:
    print("Does not exist")

Does not exist


In [29]:
#Code to query the existence of a residue position (will be iterative)
if not 17 in sample["Nuc_Pos(Ref)"]:
    print("Value does not exist")

Value does not exist


In [12]:
dels.Cluster_ID.unique()[10:15]

array(['Uniq1745', 'Uniq1748', 'Uniq1985', 'Uniq1994', 'Uniq2102'],
      dtype=object)

In [10]:
len(dels.Cluster_ID.unique())

564

In [339]:
"""
Write out substitutions
"""
#Print number of unique codes and skip a line before printing information for each code
with open(subs_report,"w") as file: 
    print("Number of unique substitutions in input: ",len(subs.code.unique()),sep="",end="\n\n",file=file)


2193

In [ ]:


#Iterator: loops through all unique codes 
for i in range(len(subs.code.unique())):
    sub_subs = subs[subs.code==subs.code.unique()[i]]
    #Create empty lists to store all cluster IDs with the variant in question, and the size of each cluster
    clusters=[]
    clust_sizes=[]
    #Loop through all entries in the dataframe that match the current cluster ID
    for item in sub_subs["Cluster_ID"]:
        #Add the cluster ID to the cluster list
        clusters.append(item)
        #Find the size of the cluster ID and add that to the list for cluster sizes
        clust_sizes.append(sub_subs.loc[sub_subs["Cluster_ID"]==item,"N_sequences_in_cluster"].iat[0])
    #Once the lists for the cluster IDs and sizes for the variant type have been created, iterate through each list and add it to the report file.
    #First, the total number of sequences with the given variant is computed and added to the file next to the variant code (variant header).
    total_seq=np.array(clust_sizes).sum()
    #Write variant header information to file
    print(">{}|{} sequences".format(subs.code.unique()[i],total_seq))
    #Iterate through the cluster names and sizes in the lists and add information to file
    for j in range(len(clusters)):
        print(clusters[j]," size "+str(clust_sizes[j]),sep=":")
    #Add a line break at the end of the information for the current code
    print("")

In [320]:
len(subs.code.unique())

2193

In [64]:
test_list=["a","-","-","-","-","-","-","-","b","c","d"]

In [73]:
#Iteration skip without recursion
def non_recursive_iterator(i_begin,test_list,end=False):
    for i in range(i_begin,len(test_list)):
        if test_list[i]!="-":
            print(test_list[i])
        elif test_list[i]=="-":
            print("found a blank at index {}".format(i))
            continue

In [74]:
non_recursive_iterator(0,test_list)

a
found a blank at index 1
found a blank at index 2
found a blank at index 3
found a blank at index 4
found a blank at index 5
found a blank at index 6
found a blank at index 7
b
c
d


In [73]:
test_list[2]!="-"

False

In [6]:
#Model for multi-base event recording
def mini_iterator(i_begin,test_list):
    for i in range(i_begin,len(test_list)):
        if test_list[i]!="-":
            print(test_list[i])
        elif test_list[i]=="-":
            print("found a blank at index {}".format(i))
            i_blank=i
            skipper(i_blank,test_list)
            break #Break the for loop (critical to prevent recording the event more than once)
            
def skipper(i_blank,test_list):
    for i in range(i_blank+1,len(test_list)):
        if test_list[i] == "-":
            continue
        elif test_list[i] != "-":
            print("blanks discovered between indeces {} and {}".format(i_blank,i))
            mini_iterator(i,test_list) 
            break
          

In [81]:
mini_iterator(0,test_list)

a
found a blank at index 1
blanks discovered between indeces 1 and 8
b
c
d


In [131]:
for n in test:
    print(n)
    if n>10:
        next(n)
        next(n)

6
22


TypeError: 'int' object is not an iterator

In [142]:
c=iter(test)
for n in c:
    print(n)
    if n>10:
        for i in range(2):
            print("next iteration")
            next(c)

6
22
next iteration
next iteration
5
2


In [86]:
test=[6,22,22,4,5,2]

In [87]:
for a in test:
    print(a)
    if a>10:
        print("%s is greater than 10" % a)
        test.remove(test[test.index(a)])

6
22
22 is greater than 10
4
5
2


In [88]:
test

[6, 22, 4, 5, 2]

In [92]:
#Carrige retrun example (for progress meter)
for x in range(100000001):
    if x%100000 == 0:
        print('{}\r'.format(x), end="")
print()

100000000
